In [62]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from Secrets import CLIENT_ID, CLIENT_SECRET, REDIRECT_URL
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from dateutil.relativedelta import relativedelta
from sklearn.metrics.pairwise import cosine_similarity

In [63]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URL,
                                               scope="user-library-read"))

df = pd.read_csv('./dataset.csv')
df = df[df.columns[1:]]
df = df.drop(columns=['time_signature', 'duration_ms'])
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',504)
pd.set_option('display.width',1000)


## Feature Engineering

In [64]:
def feature_engineer(df):
    scaler = MinMaxScaler()
    features_to_scale = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']
    features_to_OHE = ['key', 'explicit', 'track_genre', 'popularity']
    
    # Ensure scaling does not affect 'date_added'
    df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
    
    def perform_OHE(df, column):
        OHE = pd.get_dummies(df[column], drop_first=True, dtype=int, prefix=column)
        df = df.drop(columns=column, axis='columns')
        df = pd.concat([df, OHE], axis=1)
        return df

    # Modify popularity without affecting 'date_added'
    df['popularity'] = df['popularity'].apply(lambda x: x//5)

    # Perform OHE and ensure 'date_added' is retained
    for feature in features_to_OHE:
        df = perform_OHE(df, feature)
    
    # No explicit action needed for 'date_added' as long as it's not removed or altered
    
    print(df.head())
    return df

In [65]:
#6meRpNHvKC1VcBl5MIbVxo - hehe
#1Ob3QykC4dzf3rQwWxE9LV - study


playlist_id = '6meRpNHvKC1VcBl5MIbVxo'
playlist_length = sp.playlist_items(playlist_id)['total']

track_ids = []
date_added = {}


for i in range(playlist_length // 100 + 1):
    playlist = sp.playlist_tracks(playlist_id, market = 'CAN',fields='items', limit=100, offset = i * 100 )
    for index, track in enumerate(playlist['items']):
        if track['track'] is not None and track['track']['id'] is not None:
            date_added[track['track']['id']] = playlist['items'][index]['added_at']
            track_ids.append(track['track']['id'])

In [66]:
playlist_audio_data = []
artist_genres = {}

In [67]:
#Fetching track info from the Spotify API
playlist_track_audio_features = []
count = 0
for i in range(0,len(track_ids), 100):
    playlist_track_audio_features.extend(sp.audio_features(tracks = track_ids[i : i + 100]))

playlist_track_info = []
for i in range(0, len(track_ids), 50):
    playlist_track_info.extend(sp.tracks(track_ids[i : i + 50])['tracks'])

In [68]:
# Processing audio features and info for each track in the playlist
track_audio_features = [feature for feature in playlist_track_audio_features]
track_info = [info for info in playlist_track_info]
track_artist_ids = [info['artists'][0]['id'] for info in playlist_track_info]

In [69]:
#Fetching artist Genres
for i in range(0, len(track_artist_ids), 50):
    results = sp.artists(track_artist_ids[i: i + 50])
    for artist in results['artists']:
        artist_genres[artist['name']] = artist_genres.get(artist['name'], artist['genres'][0] if artist['genres'] else None)

In [70]:
#Processing and adding the audio features to the dataset

for index ,id in enumerate(track_ids):
    track_artists = [artist['name'] for artist in track_info[index]['artists']]
    main_artist_genre = artist_genres[track_artists[0]]
    audio_features_keys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    audio_data = {
        'track_id': id,
        'artists': ';'.join(track_artists),
        'album_name': track_info[index]['album']['name'],
        'track_name': track_info[index]['name'],
        'popularity': track_info[index]['popularity'],
        'explicit': track_info[index]['explicit'],
        **{key: track_audio_features[index][key] for key in audio_features_keys},
        'track_genre': main_artist_genre
    }
    df.loc[len(df.index)] = audio_data
df.tail()

,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
114196,4cg1yakyRSIOjxKM2I7J1q,Young Thug,Slime Season 3,Digits,63,True,0.769,0.632,10,-4.228,0,0.0537,0.03250,0.0000,0.1160,0.2940,131.932,atl hip hop
114197,2i2qDe3dnTl6maUE31FO7c,Lil Durk;Future,Lil Durk Presents: Loyal Bros 2,Mad Max,57,True,0.904,0.635,5,-8.744,0,0.2330,0.01740,0.0000,0.0919,0.6240,150.009,chicago drill
114198,5lFDtgWsjRJu8fPOAyJIAK,Drake,Back To Back,Back To Back,64,True,0.893,0.480,1,-3.728,0,0.3560,0.00855,0.0000,0.1160,0.3820,86.976,canadian hip hop
114199,3WRIaWsws011vHMd9uzPjG,Future;Travis Scott,Future Hndrxx Presents: The WIZRD,First Off (feat. Travis Scott),55,True,0.734,0.683,1,-6.105,1,0.0776,0.01300,0.0000,0.2800,0.4450,162.038,atl hip hop
114200,1jKXjxMWlq4BhH6f9GtZbu,Don Toliver,HARDSTONE PSYCHO,TORE UP,75,True,0.534,0.819,11,-4.329,1,0.0380,0.00120,0.0011,0.1120,0.0806,154.994,pop rap


In [71]:
print()

In [82]:

df.shape()
track_data = df.tail(playlist_length - 1).copy()

# df = df.iloc[:-100]
# track_data.head()

KeyError: 'popularity'

In [73]:
track_data['date_added'] = date_added.values()
track_data.head()

,track_id,artists,album_name,track_name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,explicit_True,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_atl hip hop,track_genre_birmingham grime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_canadian contemporary r&b,track_genre_canadian hip hop,track_genre_cantopop,track_genre_chicago drill,track_genre_chicago rap,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_conscious hip hop,track_genre_country,track_genre_dance,track_genre_dance pop,track_genre_dancehall,track_genre_dark trap,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_dfw rap,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_emo rap,track_genre_florida drill,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip hop,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie hip hop,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_lgbtq+ hip hop,track_genre_malay,track_genre_mandopop,track_genre_melodic drill,track_genre_melodic rap,track_genre_memphis hip hop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new jersey rap,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop rap,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r&b,track_genre_r-n-b,track_genre_rap,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_songwriter,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_uk hip hop,track_genre_world-music,popularity_1,popularity_2,popularity_3,popularity_4,popularity_5,popularity_6,popularity_7,popularity_8,popularity_9,popularity_10,popularity_11,popularity_12,popularity_13,popularity_14,popularity_15,popularity_16,popularity_17,popularity_18,popularity_19,popularity_20,date_added
114000,3F5CgOj3wFlRv51JsHbxhe,Drake;21 Savage,"Honestly, Nevermind",Jimmy Cooks (feat. 21 Savage),0.537056,0.673,0.829033,1,0.181347,0.000308,0.000002,0.0930,0.367839,165.921,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2022-07-07T01:18:56Z
114001,2BcMwX1MPV6ZHP4tUT9uq6,Drake;21 Savage;Project Pat,Certified Lover Boy,Knife Talk (with 21 Savage ft. Project Pat),0.861929,0.42

In [74]:
import pandas as pd
from dateutil.relativedelta import relativedelta

# Assuming track_data is already defined and feature_engineered
track_data = pd.DataFrame(track_data)
track_data['date_added'] = pd.to_datetime(track_data['date_added']).dt.date
first_date = track_data.iloc[0]['date_added']
last_date = track_data.iloc[-1]['date_added']

# Calculate total months difference for normalization
total_months = relativedelta(last_date, first_date).months + 1

# Calculate weights using linear descent
weights = []
for index, row in track_data.iterrows():
    months_difference = relativedelta(row['date_added'], first_date).months
    # Linear descent weighting
    weight = (total_months - months_difference)/ total_months
    weights.append(round(weight, 3))

track_data['weight'] = weights
track_data = track_data.drop(columns=['date_added'])
track_data.head()


,track_id,artists,album_name,track_name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,explicit_True,track_genre_afrobeat,track_genre_alt-rock,track_genre_alternative,track_genre_ambient,track_genre_anime,track_genre_atl hip hop,track_genre_birmingham grime,track_genre_black-metal,track_genre_bluegrass,track_genre_blues,track_genre_brazil,track_genre_breakbeat,track_genre_british,track_genre_canadian contemporary r&b,track_genre_canadian hip hop,track_genre_cantopop,track_genre_chicago drill,track_genre_chicago rap,track_genre_chicago-house,track_genre_children,track_genre_chill,track_genre_classical,track_genre_club,track_genre_comedy,track_genre_conscious hip hop,track_genre_country,track_genre_dance,track_genre_dance pop,track_genre_dancehall,track_genre_dark trap,track_genre_death-metal,track_genre_deep-house,track_genre_detroit-techno,track_genre_dfw rap,track_genre_disco,track_genre_disney,track_genre_drum-and-bass,track_genre_dub,track_genre_dubstep,track_genre_edm,track_genre_electro,track_genre_electronic,track_genre_emo,track_genre_emo rap,track_genre_florida drill,track_genre_folk,track_genre_forro,track_genre_french,track_genre_funk,track_genre_garage,track_genre_german,track_genre_gospel,track_genre_goth,track_genre_grindcore,track_genre_groove,track_genre_grunge,track_genre_guitar,track_genre_happy,track_genre_hard-rock,track_genre_hardcore,track_genre_hardstyle,track_genre_heavy-metal,track_genre_hip hop,track_genre_hip-hop,track_genre_honky-tonk,track_genre_house,track_genre_idm,track_genre_indian,track_genre_indie,track_genre_indie hip hop,track_genre_indie-pop,track_genre_industrial,track_genre_iranian,track_genre_j-dance,track_genre_j-idol,track_genre_j-pop,track_genre_j-rock,track_genre_jazz,track_genre_k-pop,track_genre_kids,track_genre_latin,track_genre_latino,track_genre_lgbtq+ hip hop,track_genre_malay,track_genre_mandopop,track_genre_melodic drill,track_genre_melodic rap,track_genre_memphis hip hop,track_genre_metal,track_genre_metalcore,track_genre_minimal-techno,track_genre_mpb,track_genre_new jersey rap,track_genre_new-age,track_genre_opera,track_genre_pagode,track_genre_party,track_genre_piano,track_genre_pop,track_genre_pop rap,track_genre_pop-film,track_genre_power-pop,track_genre_progressive-house,track_genre_psych-rock,track_genre_punk,track_genre_punk-rock,track_genre_r&b,track_genre_r-n-b,track_genre_rap,track_genre_reggae,track_genre_reggaeton,track_genre_rock,track_genre_rock-n-roll,track_genre_rockabilly,track_genre_romance,track_genre_sad,track_genre_salsa,track_genre_samba,track_genre_sertanejo,track_genre_show-tunes,track_genre_singer-songwriter,track_genre_ska,track_genre_sleep,track_genre_songwriter,track_genre_soul,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_uk hip hop,track_genre_world-music,popularity_1,popularity_2,popularity_3,popularity_4,popularity_5,popularity_6,popularity_7,popularity_8,popularity_9,popularity_10,popularity_11,popularity_12,popularity_13,popularity_14,popularity_15,popularity_16,popularity_17,popularity_18,popularity_19,popularity_20,weight
114000,3F5CgOj3wFlRv51JsHbxhe,Drake;21 Savage,"Honestly, Nevermind",Jimmy Cooks (feat. 21 Savage),0.537056,0.673,0.829033,1,0.181347,0.000308,0.000002,0.0930,0.367839,165.921,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
114001,2BcMwX1MPV6ZHP4tUT9uq6,Drake;21 Savage;Project Pat,Certified Lover Boy,Knife Talk (with 21 Savage ft. Project Pat),0.861929,0.424,0.738990,0,0.335751

In [75]:
start_index = 0
end_index = 1
num_rows = len(track_data.values)
num_cols = len(track_data.columns)
results_array = [0] * num_cols
for i in range(num_rows):
    row = track_data.iloc[i]
    row_weight = track_data.iloc[i, -1]
    for index, value in enumerate(row[4: num_cols]):
        results_array[index] += value / num_rows
         
print(results_array)

[0.7389347677854381, 0.6035910447761195, 0.7924895618829807, 0.4776119402985066, 0.2016446266078932, 0.16186393334532162, 0.007715551343283582, 0.1849019900497515, 0.3990369759243981, 124.83338805970149, 0.2587064676616917, 0.06467661691542287, 0.014925373134328358, 0.05472636815920397, 0.07462686567164178, 0.07960199004975124, 0.09950248756218907, 0.07462686567164178, 0.06467661691542287, 0.0845771144278607, 0.07462686567164178, 0.9402985074626836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2039800995024877, 0.004975124378109453, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.039800995024875614, 0.16417910447761203, 0.0, 0.009950248756218905, 0.044776119402985065, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.049751243781094516, 0.0, 0.0, 0.009950248756218905, 0.0, 0.004975124378109453, 0.0, 0.0, 0.0, 0.01990049751243781, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004975124378109453, 0.009950248756218905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08955223880597016, 0.0, 0.0, 0.0, 

In [81]:
track_data.iloc[:, :-1].head
# df.iloc[:, 4:].values

<bound method NDFrame.head of                       track_id                                            artists                                     album_name                                         track_name  danceability  energy  loudness  mode  speechiness  acousticness  instrumentalness  liveness   valence    tempo  key_1  key_2  key_3  key_4  key_5  key_6  key_7  key_8  key_9  key_10  key_11  explicit_True  track_genre_afrobeat  track_genre_alt-rock  track_genre_alternative  track_genre_ambient  track_genre_anime  track_genre_atl hip hop  track_genre_birmingham grime  track_genre_black-metal  track_genre_bluegrass  track_genre_blues  track_genre_brazil  track_genre_breakbeat  track_genre_british  track_genre_canadian contemporary r&b  track_genre_canadian hip hop  track_genre_cantopop  track_genre_chicago drill  track_genre_chicago rap  track_genre_chicago-house  track_genre_children  track_genre_chill  track_genre_classical  track_genre_club  track_genre_comedy  track_genre_cons

In [77]:

vectors = df.iloc[:, 4:].values
print(results_array.shape)
input_vector = np.array(results_array)
similarity_matrix = cosine_similarity(input_vector.reshape(1,-1), vectors)
similarity_array = similarity_matrix.flatten()
top_40_indices = similarity_array.argsort()[-40:][::-1]
print(top_40_indices)
top_40_vectors = vectors[top_40_indices]

print(top_40_vectors)

AttributeError: 'list' object has no attribute 'shape'